# MOwNiT lab2 - interpolacja oraz regresja liniowa

<h3> Przydatne linki: </h3>


- CPP: https://en.cppreference.com/w/

- Interpolacja Hermite'a: https://en.wikipedia.org/wiki/Cubic_Hermite_spline
- Interpolacja wielomianami: https://graphics.stanford.edu/courses/cs205a-13-fall/assets/notes/chapter11.pdf
- Interpolacja Lagrange'a: https://www.mimuw.edu.pl/~kmoszyns/cc.pdf
- Regresja liniowa -> "tutorial for dummies": https://machinelearningmastery.com/simple-linear-regression-tutorial-for-machine-learning/
- Kontynuacja powyższego: https://machinelearningmastery.com/implement-simple-linear-regression-scratch-python/

<h3> Cubic Spline Interpolation </h3>


W skrócie: Mając zbiór danych, które posiadają dany interwał (np. audio) możemy dokonać interpolacji pomiędzy dowolnymi dwoma punktami. 
Powyższa metoda poza interpolacją pomiędzy samymi punktami zapewnia również ciągłość pierwszej pochodnej (funkcja C1). Bardziej szczegółowy opis metody znajduje się w linkach powyżej, natomiast sposób w jaki dosyć łatwo jest ją zaimplementować znajduje się tutaj: https://dsp.stackexchange.com/questions/18265/bicubic-interpolation/18273#18273


In [1]:
#include <iostream>
#include <vector>
#include <array>
#include <math.h>

typedef std::vector<std::array<float, 2>> PointsList2D;

In [2]:
/// <summary>
/// Template function used for clamping specified datasets
/// </summary>
template <typename T>
T GetIndexClamped(const std::vector<T> points, int index)
{
	if (index < 0)
		return points[0];
	else if (index >= int(points.size()))
		return points.back();
	else
		return points[index];
}

In [3]:
/// <summary>
/// Defines structure for interpolation classes
/// </summary>
class IInterpolation
{
	// public construction and destruction methods
	public:
		virtual ~IInterpolation() = default;

	// public interface methods
	public:
        virtual void Interpolate1D(int pointsToInterpolate) =  0;
		virtual void Interpolate2D(int pointsToInterpolate) =  0;

};

/// <summary>
/// Defines Hermite Cubic Interpolation 
/// </summary>
class CubicInterpolation : public IInterpolation
{
	// public construction methods
	public:
		CubicInterpolation(const PointsList2D& points) : pointsList(points) {}

    // IInterpolation methods
    public:
        void Interpolate2D(int pointsToInterpolate) override
		{
			std::vector<int> index(pointsToInterpolate);
			std::vector<float> t;
			std::vector<float> tx;

			int i = 0, points_size = pointsList.size() - 1;
			std::generate(index.begin(), index.end(), [&i, &pointsToInterpolate, &points_size, &t, &tx]()
			{
				float percent = ((float)i) / (float(pointsToInterpolate - 1));
				tx.push_back((points_size)* percent);
				t.push_back(tx[i] - floor(tx[i]));
				return int(tx[i++]);
			});

			for (int i = 0; i < pointsToInterpolate; ++i)
			{
				std::array<PolynomialCoeffs, 2> coeffs;
				std::array<float, 2> A = GetIndexClamped(pointsList, index[i] - 1);
				std::array<float, 2> B = GetIndexClamped(pointsList, index[i]);
				std::array<float, 2> C = GetIndexClamped(pointsList, index[i] + 1);
				std::array<float, 2> D = GetIndexClamped(pointsList, index[i] + 2);

				for (int i = 0; i < 2; i++)
				{
					coeffs[i].A = A[i];
					coeffs[i].B = B[i];
					coeffs[i].C = C[i];
					coeffs[i].D = D[i];
				}

				float x = CubicHermite(coeffs[0], t[i]);
				float y = CubicHermite(coeffs[1], t[i]);

				std::cout << "Value at " << tx[i] << " = " << x << "  " << y << std::endl;
			}
		};

        void Interpolate1D(int pointsToInterpolate) override 
        {
            // TODO
        }
        
    // private methods
	private:
		struct PolynomialCoeffs
		{
			float A, B, C, D;
		};

        float CubicHermite(PolynomialCoeffs coeffs, float t) const
		{
			float a = -coeffs.A / 2.0f + (3.0f*coeffs.B) / 2.0f - (3.0f*coeffs.C) / 2.0f + coeffs.D / 2.0f;
			float b = coeffs.A - (5.0f*coeffs.B) / 2.0f + 2.0f*coeffs.C - coeffs.D / 2.0f;
			float c = -coeffs.A / 2.0f + coeffs.C / 2.0f;
			float d = coeffs.B;

			return a * pow(t, 3) + b * pow(t, 2) + c * t + d;
		}
        
	// private members
	private:
		const PointsList2D& pointsList;

};


// const PointsList1D points1D =
// {
//     0.0f,
//     1.6f,
//     2.3f,
//     3.5f,
//     4.3f,
//     5.9f,
//     6.8f
// };

const PointsList2D points2D =
{
    { 0.0f, 1.1f },
    { 1.6f, 8.3f },
    { 2.3f, 6.5f },
    { 3.5f, 4.7f },
    { 4.3f, 3.1f },
    { 5.9f, 7.5f },
    { 6.8f, 0.0f }
};

auto interpolation = CubicInterpolation(points2D);
interpolation.Interpolate2D(13);


Value at 0 = 0  1.1
Value at 0.5 = 0.75625  4.8125
Value at 1 = 1.6  8.3
Value at 1.5 = 1.975  7.9625
Value at 2 = 2.3  6.5
Value at 2.5 = 2.89375  5.5875
Value at 3 = 3.5  4.7
Value at 3.5 = 3.875  3.5125
Value at 4 = 4.3  3.1
Value at 4.5 = 5.09375  5.66875
Value at 5 = 5.9  7.5
Value at 5.5 = 6.45  4.025
Value at 6 = 6.8  0


Wizualizacja powyższej metody: http://demofox.org/cubichermite2d.html

**Zadanie 1**
W powyższym przykładzie zaimplementować metodę Interpolate1D (najprościej w sposóby analogiczny do metody Interpolate2D). 

(**Cpp**\*) Zmienić implementację template'a w taki sposób, aby przyjmował vector punktów (np. PointsList2D) i zwracał również taki wektor, ale odpowiednio "docięty" (ang. Clamped vector/list).   

**Zadanie 2**
Zaimplementować metodę interpolacji Lagrange'a wpasowując sie do powyższego schematu (może być jedynie interpolacja 1D). Porównać obie metody (wystarczy pod kątem teoretycznym, nie implementacyjnym).

In [4]:
class LagrangeInterpolation : public IInterpolation
{
    public:
        LagrangeInterpolation(const PointsList2D& points) : pointsList(points) {}

        void Interpolate1D(int pointsToInterpolate) override
        {
            // TODO
        };
        void Interpolate2D(int pointsToInterpolate) override
        {
            // TODO
        };

    private:
        const PointsList2D& pointsList;
};

# Linear regression

**Zadanie 3** Napisz funkcję liczącą błąd średniokwadratowy. Na wejściu musi dostawać dwie tablice/dwa wektory równej długości, a na wyjściu ma zwracać sumę kwadratów różnic pomiędzy kolejnymi elementami tych wektorów.

In [5]:
float rmse(std::vector<float> x, std::vector<float> y)
{
    // remove this return when implemented
    return 0.0f;
}

**Zadanie 4** Napisz funkcję pobierającą dwa wektory float'ów i zwracającą parametry a i b prostej o równaniu y = ax + b, będącej najlepszą aproksymacją tych punktów.

In [6]:
std::pair<float,float> linearRegression (std::vector<float> x, std::vector<float> y)
{
    // remove this return when implemented
    return std::make_pair(0.2f, 0.3f);   
}

**Zadanie 5** Wynik najlepiej przedstawić na wykresie w postaci umieszczenia na nim punktów oraz dopasowanej do nich prostej (najłatwiej dane z C++ zrzucić do pliku lub po prostu skopiować z konsoli, a następnie wykres sporządzić za pomocą gnuplot lub matplotlib - python). Dla odważnych - można korzystać z bibliotek C++ (np. QtCharts).

(\*) **Zadanie 6** Napisz klasę enkapsulującą model regresji liniowej. Klasa powinna mieć metody:

- fit, przyjmującą punkty, do których będziemy dopasowywać model,
- predict, przyjmująca wektor floatów (tylko x) i zwracającą predykcje naszego modelu dla tych danych wejściowych,
- pole coeffs, zwracają współczynniki prostej, którą dopasowywaliśmy

Model można zweryfikować na dowolnych danych dwuwymiarowych (np. [tych](https://www.math.muni.cz/~kolacek/docs/frvs/M7222/data/AutoInsurSweden.txt)). Wynik przedstaw analogicznie do tego w zadaniu 5.

In [7]:
class LinearRegressor
{
    public:
        LinearRegressor() { }
        float fit(std::vector<float> x, std::vector<float> y) 
        { 
            // remove this return when implemented
            return 0.0f;
        }
        std::vector<float> predict(std::vector<float> x) 
        { 
            // remove this return when implemented
            return std::vector<float> {0.2f, 0.3f}; 
        }

    private:
        std::pair<float, float> coefficients;
};

Przydatny [link](https://courses.washington.edu/matlab1/ModelFitting.html) do zadania dodatkowego.